In [1]:
%load_ext sql

In [2]:
%sql sqlite://

# Exercise 1

Sie möchten in einer Tabelle in einer Datenbank speichern, an welchem Datum Sie sich mit welcher Person an welchem Ort getroffen haben.

a) Geben Sie das Schema dieser Datenbank

In [3]:
%%sql

DROP TABLE IF EXISTS MEETINGS;
CREATE TABLE MEETINGS (
    NAME VARCHAR(100),
    DATE DATE
);

 * sqlite://
Done.
Done.


[]

b) Geben Sie ein Beispiel für eine Instanz dieses Schemas.

In [4]:
%%sql

INSERT INTO MEETINGS (NAME, DATE) VALUES ('Max Muster', '2024-01-01 15:00:00');
SELECT * FROM MEETINGS;

 * sqlite://
1 rows affected.
Done.


NAME,DATE
Max Muster,2024-01-01 15:00:00


# Exercise 2

a) Erweitern Sie das Beispiel aus der letzten Aufgabe: Sie wollen in der selben Tabelle auch noch die (aktuelle) Telefonnummer der Personen speichern.

In [5]:
%%sql

ALTER TABLE MEETINGS ADD COLUMN PHONE_NUMBER VARCHAR(100) NULL;
UPDATE MEETINGS SET PHONE_NUMBER = '1234567890' WHERE NAME = 'Max Muster';

 * sqlite://
Done.
1 rows affected.


[]

b) Erklären Sie anhand dieses Beispiels Redundanz in einer Datenbank

An sich können keine redundanten Treffen mit einer Person entstehen. Man kann die Person nicht zweimal am exakt gleichen Zeitpunk treffen.
Trifft man die Person jedoch mehrmals wird der Name und die Telefonnummer mehrfach hinterlegt, was zu redundanten Einträgen führt.
In unserer Datenbank haben wir Max Muster bisher einmal getroffen, wenn wir Max Muster nocheinmal treffen, ist Max Muster ein redundanter Eintrag.

In [6]:
%%sql

INSERT INTO MEETINGS (NAME, DATE, PHONE_NUMBER) VALUES ('Max Muster', '2024-01-01 16:00:00', '1234567890');
SELECT * FROM MEETINGS;

 * sqlite://
1 rows affected.
Done.


NAME,DATE,PHONE_NUMBER
Max Muster,2024-01-01 15:00:00,1234567890
Max Muster,2024-01-01 16:00:00,1234567890


Wie wir sehen können ist Max Muster zweimal hinterlegt, was zu einer Redundanz führt.

c) Erklären Sie, wie aus dieser Redundanz Inkonsistenz entstehen kann.

Nehmen wir mal an, wir treffen einen anderen Max Muster der eine andere Telefonnummer hat. Mit zwei Max Mustern ist dies noch einigermassen übersichtlich.
Ist es jedoch eine Firma die alle Meetings managed wird dies sehr schnell unübersichtlich.

In [7]:
%%sql

INSERT INTO MEETINGS (NAME, DATE, PHONE_NUMBER) VALUES ('Max Muster', '2024-02-01 16:00:00', '9234567890');
SELECT * FROM MEETINGS;

 * sqlite://
1 rows affected.
Done.


NAME,DATE,PHONE_NUMBER
Max Muster,2024-01-01 15:00:00,1234567890
Max Muster,2024-01-01 16:00:00,1234567890
Max Muster,2024-02-01 16:00:00,9234567890
